# build a database of CESM-LE files

In [12]:
import intake
import intake_cesm
import yaml

from tqdm import tqdm
import sys

import pandas as pd

import os
from subprocess import check_call


show the directories to store database and cached files (set in .config-intake-cesm.yml)

### query collection using intake plugin interface

Connect to database

In [13]:
intake_cesm_config = intake_cesm.get_options()
cache_directory = intake_cesm_config['cache_directory']
cache_directory

'/glade/scratch/mclong/intake-cesm-data'

In [14]:
build_kwargs = dict(collection_input_file='intake-collection-input-cesm1_le.yml',
                    collection_type_def_file='intake-cesm-definitions.yml',
                    overwrite_existing=False, 
                    include_cache_dir=True)

col = intake.open_cesm_metadatastore('cesm1_le', build_kwargs=build_kwargs)
col                     

Active collection: cesm1_le


<Intake catalog: cesm1_le>

In [16]:
col.df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 262092 entries, 0 to 262091
Data columns (total 18 columns):
resource            262092 non-null object
resource_type       262092 non-null object
direct_access       262092 non-null bool
experiment          262092 non-null object
case                262092 non-null object
component           262092 non-null object
stream              262092 non-null object
variable            262092 non-null object
date_range          262092 non-null object
ensemble            262092 non-null int64
files               262092 non-null object
files_basename      262092 non-null object
files_dirname       262092 non-null object
ctrl_branch_year    0 non-null float64
year_offset         34112 non-null float64
sequence_order      262092 non-null int64
has_ocean_bgc       262092 non-null bool
grid                52764 non-null object
dtypes: bool(2), float64(2), int64(2), object(12)
memory usage: 34.5+ MB


Determine which ensembles have ocean biogeochemistry variables.

In [17]:
experiments=['20C', 'RCP85']
ensembles = col.search(experiment=experiments, has_ocean_bgc=True).results.ensemble.unique().tolist()
print(ensembles)

[1, 2, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 34, 35, 101, 102, 103, 104, 105]


Determine what ocean variables are available at monthly and daily resolution.  The POP model is not smart enough to write the same variable to two different streams, so it is not necessary to inclued `stream` in database queries for POP. We can, however, view all the variables defined in each stream.

In [18]:
stream_def = {'mon': 'pop.h', 'day': ['pop.h.nday1', 'pop.h.ecosys.nday1']}

all_variables = {'mon': col.search(component='ocn', stream=stream_def['mon']).results.variable.unique().tolist(), 
             'day': col.search(component='ocn', stream=stream_def['day']).results.variable.unique().tolist()}
print(yaml.dump(all_variables))

day: [CaCO3_form_zint, DpCO2_2, ECOSYS_IFRAC_2, ECOSYS_XKW_2, FG_CO2_2, HBLT_2, HMXL_2,
  SSH_2, SST, SST2, STF_O2_2, TAUX_2, TAUY_2, WVEL_50m, XBLT_2, XMXL_2, diatC_zint_100m,
  diatChl_SURF, diazC_zint_100m, diazChl_SURF, photoC_diat_zint, photoC_diaz_zint,
  photoC_sp_zint, spC_zint_100m, spCaCO3_zint_100m, spChl_SURF, zooC_zint_100m]
mon: [ADVS, ADVS_ISOP, ADVS_SUBM, ADVT, ADVT_ISOP, ADVT_SUBM, ALK, AOU, ATM_ALT_CO2,
  ATM_CO2, BSF, CFC11, CFC12, CFC_ATM_PRESS, CFC_IFRAC, CFC_XKW, CO2STAR, CO3, CaCO3_FLUX_IN,
  CaCO3_PROD, CaCO3_form, DCO2STAR, DCO2STAR_ALT_CO2, DENITRIF, DIA_DEPTH, DIA_IMPVF_CFC11,
  DIA_IMPVF_CFC12, DIA_IMPVF_IAGE, DIA_IMPVF_SALT, DIA_IMPVF_TEMP, DIC, DIC_ALT_CO2,
  DOC, DOC_prod, DOC_remin, DOFe, DOFe_prod, DON, DON_prod, DOP, DOP_prod, DpCO2,
  DpCO2_ALT_CO2, ECOSYS_ATM_PRESS, ECOSYS_IFRAC, ECOSYS_XKW, EVAP_F, FG_ALT_CO2, FG_CO2,
  FW, Fe, Fe_scavenge, Fe_scavenge_rate, FvICE_ALK, FvICE_DIC, FvPER_ALK, FvPER_DIC,
  H2CO3, HBLT, HCO3, HDIFB_CFC11, HDIFB_CFC12, H

This analysis will focus on a subset of those variables.

In [19]:
variables = yaml.load('''
day: [ECOSYS_IFRAC_2, HMXL_2, SST, XMXL_2, 
      diatC_zint_100m, diatChl_SURF, photoC_diat_zint,
      photoC_sp_zint, spC_zint_100m, spChl_SURF, zooC_zint_100m]
mon: [ECOSYS_IFRAC, Fe, HMXL, Jint_100m_DIC, NH4, NO3, NOx_FLUX, PAR_avg, PD, PO4, 
      POC_FLUX_IN, POC_PROD, SALT, SiO3, TBLT, TEMP, XMXL, diatC, diatChl, 
      diat_Fe_lim, diat_N_lim, diat_PO4_lim, diat_SiO3_lim, diat_agg, diat_light_lim, diat_loss, 
      graze_diat, graze_diaz, graze_sp, photoC_NO3_diat, photoC_NO3_diat_zint, photoC_NO3_sp, 
      photoC_NO3_sp_zint, photoC_diat, photoC_sp, spC, spChl, sp_Fe_lim, sp_N_lim, sp_PO4_lim, 
      sp_agg, sp_light_lim, sp_loss, tend_zint_100m_NO3, zooC, zoo_loss]
''')
with open('variables.yml', 'w') as fid:
    yaml.dump(variables, fid)
    
variable_list = []
for k, v in variables.items():
    variable_list.extend(v)
    
!cat variables.yml

day: [ECOSYS_IFRAC_2, HMXL_2, SST, XMXL_2, diatC_zint_100m, diatChl_SURF, photoC_diat_zint,
  photoC_sp_zint, spC_zint_100m, spChl_SURF, zooC_zint_100m]
mon: [ECOSYS_IFRAC, Fe, HMXL, HOR_DIFF, Jint_100m_DIC, NH4, NO3, NOx_FLUX, PAR_avg,
  PD, PO4, POC_FLUX_IN, POC_PROD, SALT, SiO3, TBLT, TEMP, XMXL, diatC, diatChl, diat_Fe_lim,
  diat_N_lim, diat_PO4_lim, diat_SiO3_lim, diat_agg, diat_light_lim, diat_loss, graze_diat,
  graze_diaz, graze_sp, photoC_NO3_diat, photoC_NO3_diat_zint, photoC_NO3_sp, photoC_NO3_sp_zint,
  photoC_diat, photoC_sp, spC, spChl, sp_Fe_lim, sp_N_lim, sp_PO4_lim, sp_agg, sp_light_lim,
  sp_loss, tend_zint_100m_NO3, zooC, zoo_loss]


### query catalog for full-subset

In [20]:
cat = col.search(experiment=experiments,
                 ensemble=ensembles, 
                 variable=variable_list)
cat.results.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7134 entries, 226956 to 1379
Data columns (total 18 columns):
resource            7134 non-null object
resource_type       7134 non-null object
direct_access       7134 non-null bool
experiment          7134 non-null object
case                7134 non-null object
component           7134 non-null object
stream              7134 non-null object
variable            7134 non-null object
date_range          7134 non-null object
ensemble            7134 non-null int64
files               7134 non-null object
files_basename      7134 non-null object
files_dirname       7134 non-null object
ctrl_branch_year    0 non-null float64
year_offset         0 non-null float64
sequence_order      7134 non-null int64
has_ocean_bgc       7134 non-null bool
grid                7134 non-null object
dtypes: bool(2), float64(2), int64(2), object(12)
memory usage: 961.4+ KB


### ensure all data is accessible on spinning-disk

The catalog includes data from multiple locations; what are those locations and are they accessible via direct access?

In [21]:
df = cat.results
resource_access = {}
for res in df.resource.unique():
    resource_access[res] = df.direct_access.loc[df.resource == res].unique()[0]
resource_access

{'HPSS:hsi:/CCSM/csm/CESM-CAM5-BGC-LE': False,
 'GLADE:posix:/glade/collections/cdg/data/cesmLE/CESM-CAM5-BGC-LE': True,
 'CACHE:posix:/glade/scratch/mclong/intake-cesm-data': True}

Determine which files need to be transferred from tape.

In [22]:
df_transfer = pd.DataFrame(columns=df.columns)
for file in df.files_basename.unique():
    df_file_subset = df.loc[df.files_basename == file]
    if not any(df_file_subset.direct_access):
        df_transfer = pd.concat((df_transfer, df_file_subset), ignore_index=True) 
df_transfer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3368 entries, 0 to 3367
Data columns (total 18 columns):
resource            3368 non-null object
resource_type       3368 non-null object
direct_access       3368 non-null object
experiment          3368 non-null object
case                3368 non-null object
component           3368 non-null object
stream              3368 non-null object
variable            3368 non-null object
date_range          3368 non-null object
ensemble            3368 non-null object
files               3368 non-null object
files_basename      3368 non-null object
files_dirname       3368 non-null object
ctrl_branch_year    0 non-null object
year_offset         0 non-null object
sequence_order      3368 non-null object
has_ocean_bgc       3368 non-null object
grid                3368 non-null object
dtypes: object(18)
memory usage: 473.7+ KB


In [23]:
for idx, row in tqdm(df_transfer.iterrows(), file=sys.stdout):
    file_remote = row.files
    file_local = os.path.join(cache_directory, row.files_basename)
    check_call(['hsi',f"cget {file_local} : {file_remote}"])

284it [30:52, 191.53s/it]

KeyboardInterrupt: 

In [ ]:
col = intake_cesm.CESMCollections(collection_input_file=collection_input_file,
                                  collection_type_def_file=collection_type_def_file,
                                  overwrite_existing=True, 
                                  include_cache_dir=True)
col.df.info()

In [ ]:
%load_ext watermark

In [ ]:
%watermark --iversion -g -h -m -v -u -d